<a href="https://colab.research.google.com/github/MaryM12/Statistics_and_R_IB/blob/homework_lecture_5_6/lecture5-6/code/Molodova_homework_lecture_5_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import modules
import pandas as pd
import numpy as np

In [2]:
!gdown 1z55qjD_4bsvrtP6BdoaKqhNm11Je6CdK

Downloading...
From: https://drive.google.com/uc?id=1z55qjD_4bsvrtP6BdoaKqhNm11Je6CdK
To: /content/homework_lecture_5_data.csv
100% 174M/174M [00:01<00:00, 101MB/s]


In [3]:
# В переменную data_path надо положить путь до данных
data_path = "/content"
expression_data = pd.read_csv(f"{data_path}/homework_lecture_5_data.csv", index_col=0)
expression_data.head()


,TMCC1,RANBP3,GABRG3,ARRDC5,LRP3,TIMM23,TBPL1,BIRC8,TTC28,C19orf66,...,MAGEA6,IL4I1,LCN12,SMG6,C1orf100,WDR75,ZBTB26,SPTY2D1,PLEKHA2,Cell_type
0,33.888550,38.384340,0.044721,1.385159,11.341853,105.193306,26.463990,0.000000,1.537873,27.645407,...,21.332443,1.397932,0.000000,23.136585,0.001902,60.218548,6.307399,9.577274,44.026940,B_cell
1,16.497124,35.054090,0.314645,4.329797,8.649321,50.577618,16.684614,0.000000,1.796609,13.715611,...,1.199440,159.147130,0.259244,18.986986,0.310971,25.818224,4.193348,14.181196,35.779896,B_cell
2,5.449933,31.783958,0.002949,2.137536,1.750892,109.201050,35.265984,0.011501,0.878889,19.361662,...,0.002924,25.256378,0.000000,32.941895,0.000000,99.555990,8.323141,29.321960,30.816538,B_cell
3,33.362240,56.147026,0.037295,1.604904,6.807299,100.141620,25.446817,0.000000,2.502661,12.745437,...,0.000000,14.509516,0.063003,19.036896,0.000000,49.394980,4.655794,17.683067,47.143270,B_cell
4,50.188510,30.165264,0.039678,1.501439,1.619954,46.916790,28.053373,0.018181,1.165164,30.318027,...,0.033807,93.681190,0.031816,19.094831,0.000000,20.201437,6.935232,20.769829,53.935143,B_cell


In [4]:
b_cells_expression_data = expression_data.query("Cell_type == 'B_cell'")
nk_cells_expression_data = expression_data.query("Cell_type == 'NK_cell'")
b_cells_expression_data.shape[0]

500

In [8]:
b_cells_expression_data.iloc[:,:-1].to_csv('b_cells.csv', index=False)
nk_cells_expression_data.iloc[:,:-1].to_csv('nk_cells.csv', index=False)

In [9]:
# import modules
import pandas as pd
import numpy as np
import scipy.stats as st
from statsmodels.stats.weightstats import ztest
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_ind, mannwhitneyu

print("The tool works only with two gene expression datasets! The tables should contain ONLY numeric gene expression values.\n Each column stands for a certain gene. The column order should be the SAME in the two tables. \n")
# read paths to the files
first_cell_type_expressions_path = str(input('Enter the path to the first dataset, including the file name (i.e. a table with gene expression values for cell type 1): '))
second_cell_type_expressions_path = str(input('Enter the path to the second dataset, including the file name (i.e. a table with gene expression values for cell type 2): '))
save_results_table = str(input('Enter the path to the output (without csv extension) '))

print("Select on of the following tests to perform: ztest, ci_test, ttest_ind, mannwhitneyu ")
test = input("Select the test (mandatory!): ")

print("Choose one of the methods for testing and adjustment of pvalues: bonferroni, sidak, holm-sidak, holm, simes-hochberg, hommel, fdr_bh, fdr_by, fdr_tsbh, fdr_tsbky. \nPress Enter if no correction is needed.")
correction = input('Choose the correction method: ')

# read the data
first_table = pd.read_csv(first_cell_type_expressions_path, index_col=0)
second_table = pd.read_csv(second_cell_type_expressions_path, index_col=0)

if test == 'ztest': 
  method = ztest
elif test == 'ci_test':   
  method = 'ci_test'
elif test == 'ttest_ind':   
  method = ttest_ind
elif test == 'mannwhitneyu':   
  method = mannwhitneyu
# function that checks intervals intersection
def check_intervals_intersect(first_ci, second_ci):   
  are_intersect = first_ci[0] < second_ci[0] < first_ci[1] or second_ci[0] < first_ci[0] < second_ci[1]
  return are_intersect

# function that counts test results
def DE_tools(first_table, second_table, save_results_table, method=method, correction=None):

# lists for test results
    test_p_values, H0_rejected_list = [], []
    mean_diff = []


    for i in first_table.columns: 
    # ci test
        if method == 'ci_test':
        # for table 1
            ci_1 = st.t.interval(alpha=0.95, # 95% CI
                    df=len(first_table[i]) - 1, # df - 1
                    loc=np.mean(first_table[i]), # mean
                    scale=st.sem(first_table[i])) # se
            # for table 2
            ci_2 = st.t.interval(alpha=0.95, # 95% CI
                    df=len(second_table[i]) - 1, # df - 1
                    loc=np.mean(second_table[i]), # mean
                    scale=st.sem(second_table[i])) # se
            result = check_intervals_intersect(ci_1, ci_2)
            H0_rejected_list.append(result)

        else:
            #test
            result = method(first_table[i], second_table[i])
            H0_rejected = (result[1] < 0.05)
            H0_rejected_list.append(H0_rejected)
            test_p_values.append(result[1])

        # mean_diff
        mean_diff_each = np.mean(first_table[i]) - np.mean(second_table[i])
        mean_diff.append(mean_diff_each)

    # correction for multiple tests
    names = list(first_table.columns)
    if method != 'ci_test':
    #dictionary for df columns
        results = {
            "gene_name": names,
             test + "_p_values": test_p_values,
            "adjusted_p_values": [],
            "mean_diff": mean_diff
        }
    else:
        results = {
            "gene_name": names,
            "mean_diff": mean_diff,
            "H0_rejected": H0_rejected_list
        }
    if correction:
        adjusted_p_value = multipletests(test_p_values, method=correction)
        results["adjusted_p_values"] = adjusted_p_value[1]
        results["H0_rejected"] = adjusted_p_value[0]
    if correction == None:
        results["H0_rejected"] = H0_rejected_list



    # dataframe from dictionary
    results = pd.DataFrame(results)
    # save results to .csv
    results.to_csv(f"{save_results_table}.csv")


DE_tools(first_table, second_table, save_results_table, method=method, correction=correction)
print("Done!")

The tool works only with two gene expression datasets! The tables should contain ONLY numeric gene expression values.
 Each column stands for a certain gene. The column order should be the SAME in the two tables. 

Enter the path to the first dataset, including the file name (i.e. a table with gene expression values for cell type 1): b_cells.csv
Enter the path to the second dataset, including the file name (i.e. a table with gene expression values for cell type 2): nk_cells.csv
Enter the path to the output (without csv extension) e
Select on of the following tests to perform: ztest, ci_test, ttest_ind, mannwhitneyu 
Select the test (mandatory!): ci_test
Choose one of the methods for testing and adjustment of pvalues: bonferroni, sidak, holm-sidak, holm, simes-hochberg, hommel, fdr_bh, fdr_by, fdr_tsbh, fdr_tsbky. 
Press Enter if no correction is needed.
Choose the correction method: 
Done!
